In [1]:
from nltk.corpus import cmudict, stopwords, words
from nltk import pos_tag
import random
import csv

In [2]:
cmudict.dict()['hello']

[['HH', 'AH0', 'L', 'OW1'], ['HH', 'EH0', 'L', 'OW1']]

In [137]:
def MED(w1, w2):
    
    cost = []
#     del_cost = 1
#     add_cost = 1
#     sub_cost = 2
    
    vowels = ['A', 'E', 'I', 'O', 'U']
    voiced = ['B', 'D', 'G', 'J', 'L', 'M', 'N', 'R', 'V', 'W', 'Y', 'Z']
    unvoiced = ['C', 'F', 'H', 'K', 'P', 'S', 'T']
    
    for i in range(len(w1)+1):
        x = []
        for j in range(len(w2)+1):
            x.append(0)
        cost.append(x)
    
    for i in range(len(w1)+1):
        cost[i][0] = i
    for j in range(len(w2)+1):
        cost[0][j] = j
        
    # baseline costs
    del_cost = 1
    add_cost = 1
    sub_cost = 1
        
    for i in range(1, len(w1)+1):
        for j in range(1, len(w2)+1):
            # delete cost for vowels should be high
            if w1[i-1][0] in vowels:
                del_cost = 2
                sub_cost = 4
            # add cost for vowels should be high
            if w2[j-1][0] in vowels:
                add_cost = 2
                sub_cost = 4
            # if they are both vowels, sub should be lower than add + del, but still high
            if w1[i-1][0] in vowels and w2[j-1][0] in vowels:
                sub_cost = 2
            # if they are the same sound, the sub cost should be 0
            if w1[i-1] == w2[j-1]:
                sub_cost = 0
            # get the totals
            del_total = cost[i-1][j] + del_cost
            add_total = cost[i][j-1] + add_cost
            sub_total = cost[i-1][j-1] + sub_cost
            # choose the lowest cost from the options
            options = [del_total, add_total, sub_total]
            options.sort()
            cost[i][j] = options[0]
           
#     print(w1)
#     print(w2)
#     for row in cost:
#         print(row)
    return cost[-1][-1]

In [168]:
w = 'banana'
w2 = 'santa'
res = MED(cmudict.dict()[w][0], cmudict.dict()[w2][0])

In [169]:
res

4

In [140]:
real_list = []
real_dict = []
with open('real_words_lower.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        real_list.append((row[0], eval(row[1])))
        real_dict.append(row[0])

In [141]:
real_dict = set(real_dict)

In [142]:
w = 'like'
word1 = cmudict.dict()[w][0]
options = []
for entry in real_list:
    price = MED(word1, entry[1])
    if price < 3 and w not in entry[0]:
        options.append(entry)

In [143]:
len(options)

36

In [147]:
options[:10]

[('bike', ['B', 'AY1', 'K']),
 ('dike', ['D', 'AY1', 'K']),
 ('dyke', ['D', 'AY1', 'K']),
 ('fike', ['F', 'AY1', 'K']),
 ('fyke', ['F', 'AY1', 'K']),
 ('hike', ['HH', 'AY1', 'K']),
 ('ike', ['AY1', 'K']),
 ('lac', ['L', 'AE1', 'K']),
 ('lack', ['L', 'AE1', 'K']),
 ('lai', ['L', 'AY1'])]

In [170]:
def main():
    phrase = input('Enter a phrase: ').lower()
    word_list = phrase.split()
    
    choice_list = []
    for word in word_list:
        if word in stopwords.words('english'):
            choice_list.append([])
        elif word in real_dict:
            choices = []
            pos1 = pos_tag([word])[0][1]
            w1 = cmudict.dict()[word][0]
            for entry in real_list:
                pos2 = pos_tag([entry[0]])[0][1]
                if pos1 == pos2:
                    price = MED(w1, entry[1])
                    if price < 3 and word not in entry[0] and entry[0] not in word:
                        choices.append(entry[0])
            choice_list.append(choices)
        else:
            choice_list.append([])
                
    # check to see if any subs were found:
    existing_choice = False
    for i in choice_list:
        if i:
            existing_choice = True
            break
    # if subs are available, sub each possible word
    if existing_choice:
        for j in range(len(word_list)):
            if choice_list[j]:
                new_word = random.choice(choice_list[j])
                word_list[j] = new_word
        print(' '.join(word_list))
    else:
        print('No suitable substitution found')

In [173]:
main()

Enter a phrase: i wonder if the train i need is running
i blunder if the dane i brede is gunning
